In [1]:
import empyrical as ep
import pandas as pd 
import datetime
import numpy as np
from matplotlib import pyplot as plt
import matplotlib as mpl
import matplotlib.pylab as pylab
import sys, os
import seaborn as sns
from plotly import graph_objects as go

%matplotlib inline
mpl.rcParams['figure.dpi']= 300
pd.set_option('display.float_format', lambda x: '%.3f' % x)
sns.set_style("whitegrid")
sns.set_palette("Paired")
pylab.rcParams['figure.figsize'] = 16, 6


In [2]:
portfolio = {

    "guenos/SYO.CSV":1, # ruta/ficherodarwin.csv : nivel de apalancamiento del mismo.
    "guenos/THA.csv":1,
    "guenos/ZVQ.csv":1,
    "guenos/LVS.csv":1
}


In [3]:
df_sessionlogs = {}

for informe_path in portfolio:
    multiplicador = portfolio[informe_path]
    df = pd.read_csv(informe_path,parse_dates=True,index_col=['Date'])
    df["Price"] = df["Price"] * multiplicador
    df_sessionlogs[informe_path] = df

In [4]:
df_portfolio = None

for informe_path in df_sessionlogs:
    if df_portfolio is None:
        df_portfolio = df_sessionlogs[informe_path]
        df_portfolio.columns = [informe_path]
    else:
        df_portfolio = df_portfolio.join(df_sessionlogs[informe_path].rename(columns={
            "Price":informe_path
        })).fillna(0)
        
# Creamos la columna total con el PyG del portfolio
df_portfolio["total"] = df_portfolio.sum(axis=1)

In [5]:
# Calculamos la peor sesion
peor_sesion = df_portfolio["total"].min()

# Calculamos el peor DD
cumulative = df_portfolio["total"].cumsum().values
max_return = np.fmax.accumulate(cumulative)
peor_drawdown = np.nanmin(cumulative - max_return)

# Utilizamos una formula "inventada" para llegar a un capital minimo aceptable. 
capital_sugerido = abs(round(min(5*peor_drawdown,15*peor_sesion),0))

# Creamos el DF con la informacion necesaria, y lo mostramos
df_estadisticas = pd.DataFrame.from_dict({
    "Peor Sesion": peor_sesion,
    "Peor Drawdown": peor_drawdown,
    "Capital Necesario": capital_sugerido
},"index")

df_estadisticas.columns = ["Darwins Buenos"]

In [6]:
df_estadisticas

,Darwins Buenos
Peor Sesion,-45.570
Peor Drawdown,-117.900
Capital Necesario,684.000
